## PyTorch Resnets

In [10]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH
)
from experiments.utils.parser import Parser

In [11]:
series = 80
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = 'resnet-human'
series_path = os.path.join(
    NODE_PROFILING_RESULTS_PATH,
    'series',
    str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper,
    model_name=model_name)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [12]:
# results = loader.read_results(selected)
# final_dataframe = []
# for experiment_id, result in results.items():
#     processed_exp = {'experiment_id': int(experiment_id)}
#     flattened_results = self.flatten_results(
#         results[str(experiment_id)]['responses'])
#     if log is not None:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log[experiment_id])
#     else:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log)     

In [13]:
results.columns

Index(['experiment_id', 'client_to_model_latencies_avg',
       'client_to_model_latencies_p99', 'client_to_model_latencies_p50',
       'client_to_model_latencies_var', 'client_to_model_latencies_max',
       'client_to_model_latencies_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_latencies_avg', 'model_to_client_latencies_p99',
       'model_to_client_latencies_p50', 'model_to_client_latencies_var',
       'model_to_client_latencies_max', 'model_to_client_latencies_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_coun

In [14]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1'],
    'data_type': 'image',
    'max_batch_size': ['1', '4', '16'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['resnet18'],
    'node_name': 'resnet-human',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1, 2],
    'series': 80,
    'series_meta': 'test for the queue 1,\\n cpu type: Intel(R) Xeon(R) Gold '
                   '6126 CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 300, 'loads_to_test': [10, 20]},
    'workload_type': 'static'}


In [15]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,resnet-human,resnet18,1,10Gi,1,1,10,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
1,2,video,resnet-human,resnet18,1,10Gi,1,1,20,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
2,3,video,resnet-human,resnet18,1,10Gi,1,1,10,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1
3,4,video,resnet-human,resnet18,1,10Gi,1,1,20,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1
4,5,video,resnet-human,resnet18,1,10Gi,4,1,10,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
5,6,video,resnet-human,resnet18,1,10Gi,4,1,20,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
6,7,video,resnet-human,resnet18,1,10Gi,4,1,10,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1
7,8,video,resnet-human,resnet18,1,10Gi,4,1,20,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1
8,9,video,resnet-human,resnet18,1,10Gi,16,1,10,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
9,10,video,resnet-human,resnet18,1,10Gi,16,1,20,300,80,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1


In [19]:
experiment_ids = key_config_df['experiment_id'].tolist()
metadata_columns = ['model_variant', 'max_batch_size', 'cpu_request', 'load', 'replicas']
results_columns = ['model_latencies_min', 'model_latencies_p99', 'cpu_usage_count_avg', 'model_latencies_avg', 'throughput_max']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
output['expected_throughput'] = 1/output['model_latencies_avg'] * output['max_batch_size']
# output.sort_values(by='throughput')
display(output)

,model_variant,max_batch_size,cpu_request,load,replicas,model_latencies_min,model_latencies_p99,cpu_usage_count_avg,model_latencies_avg,throughput_max,expected_throughput
0,resnet18,1,1,10,1,0.044558,0.073364,75.904711,0.053039,10.000000,18.854139
1,resnet18,1,1,20,1,0.044500,0.064762,137.838234,0.051156,18.977778,19.547972
2,resnet18,1,1,10,2,0.044381,0.081685,41.632421,0.057757,5.000000,17.313795
3,resnet18,1,1,20,2,0.044822,0.077426,76.008510,0.053532,10.000000,18.680486
4,resnet18,4,1,10,1,NaN,NaN,8.148739,NaN,10.000000,NaN
5,resnet18,4,1,20,1,NaN,NaN,10.844021,NaN,20.000000,NaN
6,resnet18,4,1,10,2,0.100578,0.102145,6.948620,0.101369,5.003344,39.459647
7,resnet18,4,1,20,2,NaN,NaN,9.119153,NaN,10.000000,NaN
8,resnet18,16,1,10,1,NaN,NaN,7.999650,NaN,10.000000,NaN
9,resnet18,16,1,20,1,NaN,NaN,12.034662,NaN,20.006689,NaN


In [17]:
results

,experiment_id,client_to_model_latencies_avg,client_to_model_latencies_p99,client_to_model_latencies_p50,client_to_model_latencies_var,client_to_model_latencies_max,client_to_model_latencies_min,model_latencies_avg,model_latencies_p99,model_latencies_p50,...,memory_usage_p50,memory_usage_var,memory_usage_max,memory_usage_min,throughput_avg,throughput_p99,throughput_p50,throughput_var,throughput_max,throughput_min
0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,343511040.0,1.191438e+16,391208960.0,3108864.0,9.752732,19.839078,9.744619,35.420347,20.000000,0.003757
1,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,267407360.0,6.148083e+15,292745216.0,2912256.0,2.427035,4.938595,2.430349,2.213757,4.986622,0.004072
2,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,325214208.0,2.299457e+15,401260544.0,230158336.0,4.862159,9.904424,4.856030,8.845399,10.000000,0.003443
3,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,412672000.0,1.589649e+16,438317056.0,3858432.0,9.768304,19.857366,9.761536,35.457490,20.006689,0.005013
4,4,0.219573,0.425863,0.231493,1.427066e-02,0.488147,0.003653,0.053532,0.077426,0.050986,...,249108480.0,2.226364e+14,249462784.0,199159808.0,4.906771,9.904348,4.905515,8.714059,10.000000,0.000000
5,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,281317376.0,1.161441e+15,330674176.0,219873280.0,4.857349,9.896656,4.853319,8.857649,10.000000,0.005477
6,1,0.170503,0.366243,0.173208,9.357890e-03,0.405669,0.004161,0.053039,0.073364,0.050855,...,239603712.0,3.036004e+12,240656384.0,234590208.0,4.894229,9.907291,4.891947,8.810392,10.000000,0.008997
7,3,0.054486,0.159764,0.053295,1.924522e-03,0.219828,0.003193,0.057757,0.081685,0.053785,...,243654656.0,1.310004e+12,244764672.0,240758784.0,2.447597,4.955184,2.446416,2.202796,5.000000,0.000000
8,7,1.005592,1.005668,1.005592,5.893753e-09,1.005669,1.005516,0.101369,0.102145,0.101369,...,253198336.0,1.198899e+14,273293312.0,235192320.0,2.446792,4.968696,2.443680,2.214690,5.003344,0.003902
9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,290050048.0,7.895669e+15,339738624.0,3239936.0,4.881228,9.900602,4.878757,8.861767,10.000000,0.006407
